4장 : 신경망 시작하기 : 분류와 회귀

In [ ]:
#4-1. 영화 리뷰 분류 : 이진 분류 문제
#목록화 IMDB dataset 수입
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
 # top 10,000 most frequently occurring words in the training data

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

In [ ]:
max([max(sequence) for sequence in train_data])

9999

In [ ]:
# 단어들에 대한 인덱스 매핑
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])

1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
print(sorted(reverse_word_index.items()))

In [ ]:
#4-1. 영화 리뷰 분류 : 이진 분류 문제
decoded_review = ''.join( # i의 값이 3보다 작으면 '?'로 대체
                         [reverse_word_index.get(i - 3, '?') for i in train_data[0]])
print(train_data[11])

[1, 54, 13, 1610, 14, 20, 13, 69, 55, 364, 1398, 21, 54, 13, 219, 12, 13, 1706, 15, 4, 20, 16, 329, 6, 176, 329, 74, 51, 13, 873, 4, 156, 71, 78, 4, 7412, 322, 16, 31, 7, 4, 249, 4, 65, 16, 38, 379, 12, 100, 157, 18, 6, 910, 20, 549, 18, 4, 1496, 21, 14, 31, 9, 24, 6, 212, 12, 9, 6, 1322, 991, 7, 3002, 4, 425, 9, 73, 2218, 549, 18, 31, 155, 36, 100, 763, 379, 20, 103, 351, 5308, 13, 202, 12, 2241, 5, 6, 320, 46, 7, 457]


In [ ]:
# imdb 데이터셋
# 데이터 준비 - 멀티-핫 벡터 변환
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
      results[i,] = 1.
  return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
# 데이터 준비 - 샘플 변경
x_train[0]

# 레이블은 쉽게 벡터로 바꿀 수 있음
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# 이것으로, 신경망에 주입할 데이터가 준비되었음

In [ ]:
# 신경망 모델 만들기
# Dense 층을 쌓을 때 두 가지 중요한 구조상의 결정이 필요함
# 1. 얼마나 많은 층을 사용할 것인가?
# 2. 각 층에 얼마나 많은 유닛을 둘 것인가?

⚫ 신경망 모델 만들기
⚫ 다음 구조를 사용
 16개의 유닛을 가진 2개의 중간층
현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층

dense 1(units = 16) - dense 2(units = 16) - dense 3(units = 1) = 확률 출력

In [ ]:
#4-4 모델 정의하기
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# 16개의 유닛이 있다는 것은 가중치 행렬 W의 크기가 (input_dimension, 16)
# 유닛을 늘리면(표현 공간을 더 고차원으로 만들면) 모델이 더욱 복잡한 표현을
# 학습할 수 있지만 계산 비용이 커지고 원하지 않는 패턴을 학습할 수도
# 있음(overfitting)
#  마지막으로 손실 함수와 옵티마이저를 선택
# 이진 분류 문제이고 모델의 출력이 확률이기 때문에(출력은 단일 노드로
# 시그모이드 활성화 함수를 사용), binary_crossentropy 손실이 적합
# mean_squared_error도 사용할 수 있음
# 확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선의 선택
# 크로스엔트로피(crossentropy)는 정보 이론(information theory) 분야에서 온
# 개념으로 확률 분포 간의 차이를 측정
# 여기에서는 원본 분포와 예측 분포 사이를 측정


# 중간층은 활성화 함수로 relu를 사용하고 마지막 층은 0과 1 사이의 확률을
# 출력하기 위해 시그모이드 활성화 함수를 사용
# 1.‘1’일 가능성이 높다는 것은 그 리뷰가 긍정일 가능성이 높다는 것을 의미 relu는
# 음수를 0으로 만드는 함수(그림 4-2)
# 2. 시그모이드는 임의의 값을 [0, 1] 사이로 출력 값을 확률처럼 해석할 수 있음


In [ ]:
#4-5. 모델 컴파일하기
model.compile(optimizer="rmsprop",
              loss="binary_corssentropy",
              metrics=["accuracy"])

In [ ]:
#4-6. 검증 세트 준비하기
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
# training your model
model.compile(optimizer='rmsprop',
    loss='binary_crossentropy', metrics=['acc'])
history = model.fit(partial_x_train, partial_y_train,
        epochs=20, batch_size=512,
        validation_data=(x_val, y_val))

history_dict = history.history # model.fit 훈련정보를 dictionary에 반환
history_dict.keys()

Epoch 1/20
30/30 [==============================] - 3s 83ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 2/20
30/30 [==============================] - 1s 35ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 3/20
30/30 [==============================] - 1s 32ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 4/20
30/30 [==============================] - 1s 36ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 5/20
30/30 [==============================] - 1s 33ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 6/20
30/30 [==============================] - 1s 35ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 7/20
30/30 [==============================] - 1s 32ms/step - loss: 0.6931 - acc: 0.5035 - val_loss: 0.6932 - val_acc: 0.4947
Epoch 8/20
30/30 [==============================] - 1s 32ms/step - loss: 0.6931 - a

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [5]:
# 4-10. 처음부터 다시 모델링하기
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
      results[i,] = 1.
  return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test) # x_test는 테스트 손실, y_test는 테스트 정확도
results

Epoch 1/4
49/49 [==============================] - 2s 25ms/step - loss: 0.7999 - accuracy: 0.5028
Epoch 2/4
49/49 [==============================] - 1s 23ms/step - loss: 0.6932 - accuracy: 0.4981
Epoch 3/4
49/49 [==============================] - 1s 27ms/step - loss: 0.6932 - accuracy: 0.4998
Epoch 4/4
782/782 [==============================] - 2s 3ms/step - loss: 0.6931 - accuracy: 0.5000


[0.6931466460227966, 0.5]